In [ ]:
import xarray as xr
import proplot as pplt
import pandas as pd
import nwp_ensemble as se
import ruamel.yaml as yaml
from utils.files import OIFSEnsemblePreprocessor, NEMOEnsemblePreprocessor
import numpy as np
import user_context as context

In [ ]:
oifs_preprocessor = OIFSEnsemblePreprocessor(pd.Timedelta(-7, "h"))
nemo_preprocessor = NEMOEnsemblePreprocessor(np.timedelta64(-7, "h"))

In [ ]:
plotting_dir = context.plotting_dir / "ensemble"
plotting_dir.mkdir(exist_ok=True)

In [ ]:
max_iters = se.max_iters
experiment_directories = []
for date_dir in se.ensemble_directory.glob("*"):
    for init_dir in date_dir.glob("*"):
        for experiment_dir in init_dir.glob("*"):
            experiment_directories.append(experiment_dir)

In [ ]:
progvars_ensemble = [
    experiment_dir / "progvar.nc" for experiment_dir in experiment_directories
]
diagvars_ensemble = [
    experiment_dir / "diagvar.nc" for experiment_dir in experiment_directories
]
nemo_t_ensemble = [
    next(experiment_dir.glob("*_grid_T.nc"))
    for experiment_dir in experiment_directories
]
setup_dicts = []
for experiment_dir in experiment_directories:
    with open(experiment_dir / "setup_dict.yaml") as yaml_file:
        setup_dicts.append(yaml.load(yaml_file, Loader=yaml.Loader))

In [ ]:
prog_ensemble = xr.open_mfdataset(
    progvars_ensemble, preprocess=oifs_preprocessor.preprocess_ensemble
)

In [ ]:
diag_ensemble = xr.open_mfdataset(
    diagvars_ensemble, preprocess=oifs_preprocessor.preprocess_ensemble
)

In [ ]:
nemo_ensemble = xr.open_mfdataset(
    nemo_t_ensemble, preprocess=nemo_preprocessor.preprocess_ensemble
)

In [ ]:
colors = ["m", "c", "y", "k"]
cpl_schemes = ["parallel", "atm-first", "oce-first", "converged SWR"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
markers = [".", "x", "1", "+"]

# OpenIFS

## Prognostic variables

Look at $T, u, v, q$ on lowest model level

In [ ]:
progvar_forecast_mean = prog_ensemble.sel(time=pd.Timedelta(2, "D")).mean(
    "initial_condition"
)
progvar_forecast_mean = progvar_forecast_mean.assign_coords(
    start_date=progvar_forecast_mean.start_date + progvar_forecast_mean.time
)
progvar_forecast_mean = progvar_forecast_mean.rename(start_date="end_date")

In [ ]:
progvar_forecast_std = prog_ensemble.sel(time=pd.Timedelta(2, "D")).std(
    "initial_condition"
)
progvar_forecast_std = progvar_forecast_std.assign_coords(
    start_date=progvar_forecast_std.start_date + progvar_forecast_std.time
)
progvar_forecast_std = progvar_forecast_std.rename(start_date="end_date")

### Temperature

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_mean.t.sel(nlev=60, coupling_scheme=cpl_scheme) - 273.15,
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Temperature on Level 60 after 2 Days",
    ylabel="Temperature [°C]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_10t.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_std.t.sel(nlev=60, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of Temperature on Level 60 after 2 Days",
    ylabel="Temperature [°C]",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_10t.pdf")

In [ ]:
for cpl_scheme in cpl_schemes:
    print(cpl_scheme)
    print(
        progvar_forecast_std.t.sel(nlev=60, coupling_scheme=cpl_scheme)
        .sum()
        .load()
        .data[()]
    )

### Wind

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")


for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_mean.u.sel(nlev=60, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Zonal Wind on Level 60 after 2 Days",
    ylabel=r"Zonal Wind $[m\; s^{-1}]$",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_10u.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_mean.v.sel(nlev=60, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Meridional Wind on Level 60 after 2 Days",
    ylabel=r"Meridional Wind $[m\; s^{-1}]$",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_10v.pdf")

### Humidity

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_mean.q.sel(nlev=60, coupling_scheme=cpl_scheme) * 1e3,
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Humidity on Level 60 after 2 Days",
    ylabel=r"Humidity $[g\; kg^{-1}]$",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_10q.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        progvar_forecast_std.t.sel(nlev=60, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of Humidity on Level 60 after 2 Days",
    ylabel=r"Humidity $[g\; kg^{-1}]$",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_10q.pdf")

## Diagnostic Variables

Look at:
- cloud cover
- boundary layer height
- convective flux of $s$, turbulent flux of $s$

In [ ]:
diagvar_forecast_mean = diag_ensemble.sel(time=pd.Timedelta(2, "D")).mean(
    "initial_condition"
)
diagvar_forecast_mean = diagvar_forecast_mean.assign_coords(
    start_date=diagvar_forecast_mean.start_date + diagvar_forecast_mean.time
)
diagvar_forecast_mean = diagvar_forecast_mean.rename(start_date="end_date")

In [ ]:
diagvar_forecast_std = diag_ensemble.sel(time=pd.Timedelta(2, "D")).std(
    "initial_condition"
)
diagvar_forecast_std = diagvar_forecast_std.assign_coords(
    start_date=diagvar_forecast_std.start_date + diagvar_forecast_std.time
)
diagvar_forecast_std = diagvar_forecast_std.rename(start_date="end_date")

### Cloud Cover

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_mean.total_cloud.sel(coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Total Cloud Cover after 2 Days",
    ylabel="Cloud Cover [-]",
    xlabel="Time of Forecast",
    ylim=[0, 1.05],
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_total_cloud.pdf")

### Boundary Layer Height

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_mean.pbl_height.sel(coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Boundary Layer Height after 2 Days",
    ylabel="Boundary Layer Height [m]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_pbl_height.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_std.pbl_height.sel(coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of Boundary Layer Height after 2 Days",
    ylabel="Boundary Layer Height [m]",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_pbl_height.pdf")

### Turbulent Flux of s

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_mean.turb_flx_s.sel(nlevp1=61, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Mean Surface Turbulent Flux of s after 2 Days",
    ylabel=r"Turbulent Flux of s [$W m^{-2}$]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_sfc_turb_flx_s.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_std.turb_flx_s.sel(nlevp1=61, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of Surface Turbulent Flux of s after 2 Days",
    ylabel=r"Turbulent Flux of s [$W m^{-2}$]",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_sfc_turb_flx_s.pdf")

### Convective Flux of s

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_mean.conv_flx_s.sel(nlevp1=61, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Mean Surface Convective Flux of s after 2 Days",
    ylabel=r"Convective Flux of s [$W m^{-2}$]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_sfc_conv_flx_s.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_std.conv_flx_s.sel(nlevp1=61, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of Surface Convective Flux of s after 2 Days",
    ylabel=r"Convective Flux of s [$W m^{-2}$]",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_sfc_conv_flx_s.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        diagvar_forecast_mean.sw_rad_flux.sel(nlevp1=61, coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Mean Surface Radiative Flux after 2 Days",
    ylabel=r"Surface Radiative Flux [$W m^{-2}$]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "sfc_radiation.pdf")

# NEMO

In [ ]:
nemo_forecast_mean = nemo_ensemble.isel(time=-1).mean("initial_condition")
nemo_forecast_mean = nemo_forecast_mean.assign_coords(
    start_date=nemo_forecast_mean.start_date + nemo_forecast_mean.time
)
nemo_forecast_mean = nemo_forecast_mean.rename(start_date="end_date")

In [ ]:
nemo_forecast_std = nemo_ensemble.isel(time=-1).std("initial_condition")
nemo_forecast_std = nemo_forecast_std.assign_coords(
    start_date=nemo_forecast_std.start_date + nemo_forecast_std.time
)
nemo_forecast_std = nemo_forecast_std.rename(start_date="end_date")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")

for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        nemo_forecast_mean.sosstsst.sel(coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title="Mean Sea Surface Temperature after 2 Days",
    ylabel="Temperature [°C]",
    xlabel="Time of Forecast",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "mean_sst.pdf")

In [ ]:
fig, ax = pplt.subplots(width="50em", height="30em")


for i in range(len(colors)):
    color = colors[i]
    cpl_scheme = cpl_schemes[i]
    marker = markers[i]
    label = labels[i]
    ax.scatter(
        nemo_forecast_std.sosstsst.sel(coupling_scheme=cpl_scheme),
        marker=marker,
        label=label,
        color=color,
    )
ax.format(
    title=r"$\sigma$ of SST after 2 Days",
    ylabel="Temperature [°C]",
    xlabel="Time of Forecast",
    yscale="log",
    yformatter="sci",
)
ax.legend(ncols=1, framealpha=1)
fig.savefig(plotting_dir / "std_sst.pdf")